In [2]:
from dataset import *
import wandb
import torchvision.transforms as transforms
import torch.optim as optim
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor, FasterRCNN
from torchvision.models.detection.backbone_utils import resnet_fpn_backbone
import pandas as pd
import numpy as np

## Hyperparameters

In [3]:
### HYPERPARAM

batchsize = 1
in_dim = (300,300)
normalization_data = torch.load('mean-std.pt')
num_classes = 7

## Data Loading

In [4]:
%load_ext autoreload
%autoreload 2
import dataset

In [5]:
dataset = dataset.SkinData('/', 'final.csv', transform=transforms.Compose([Resizing(in_dim), ToTensor, Normalizer(normalization_data)]))
train_data, test_data = torch.utils.data.random_split(dataset,[int(0.8 * len(dataset)), int(0.2 * len(dataset))],  generator=torch.Generator().manual_seed(42))
data_loader_train = torch.utils.data.DataLoader(train_data, batch_size=batchsize)
data_loader_test = torch.utils.data.DataLoader(test_data, batch_size=batchsize)

## Model

In [6]:
def fasterrcnn_resnet101_fpn(pretrained=False, progress=True,
                            num_classes=num_classes, pretrained_backbone=True,
                             trainable_backbone_layers=3, **kwargs):
    assert trainable_backbone_layers <= 5 and trainable_backbone_layers >= 0
    # dont freeze any layers if pretrained model or backbone is not used
    if not (pretrained or pretrained_backbone):
        trainable_backbone_layers = 5
    if pretrained:
        # no need to download the backbone if pretrained is set
        pretrained_backbone = False
    backbone = resnet_fpn_backbone('resnet50', pretrained_backbone)
    model = FasterRCNN(backbone, num_classes, **kwargs)
    return model

In [7]:
def initialize_model():
    model = fasterrcnn_resnet101_fpn(pretrained=False)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, 2)
    return model

In [8]:
model = initialize_model()

## Debug

In [1]:
dataset[0][1]

NameError: name 'dataset' is not defined

In [ ]:
model(dataset[0][0].unsqueeze(0), dataset[0][1])